<a href="https://colab.research.google.com/github/xinihe/jrxy_portfolio_management/blob/master/Code/tushare_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tushare
!pip install sqlalchemy
!pip install pymysql

In [ ]:
"""
Created on 6 April 2021  @author: Dai Yihang
"""
import pandas as pd
import numpy as np
import tushare as ts
import random
import time
from sqlalchemy import create_engine

token_poor = ['token']

def get_tushare_data(token_poor,function_name,retry_count=3,pause=2,**kwargs):
    '''
    从tushare导入数据的重试函数
    pro:设置完token的tushare内置的pro对象
    function_name:pro对象内置的请求数据的函数名，用字符串表示
    retry_count：重试次数
    pause：请求失败后的暂停时间
    **kwargs：给function_name函数传入的参数
    '''
    for _ in range(retry_count):
        try:
            pro = ts.pro_api(random.choice(token_poor))
            data = getattr(pro,function_name)(**kwargs)
        except:
            time.sleep(pause)
        else:
            return data
        
con = create_engine('mysql+pymysql://username:password@IP/test')

In [ ]:
code_df = get_tushare_data(token_poor,'index_weight',index_code='399300.SZ', start_date='20210101', end_date='20210402')
code_list = list(set(code_df['con_code'].tolist()))
code_list[:5]

['600436.SH', '601990.SH', '300142.SZ', '600297.SH', '300676.SZ']

In [ ]:
# code=code_list[0]
import tqdm
for code in tqdm.tqdm(code_list):
    daily_df = get_tushare_data(token_poor,'daily',ts_code=code, start_date='20190101', end_date='20210101')
    daily_df.to_sql(code[:6],con,if_exists='append',index=False)

100%|████████████████████████████████████████| 300/300 [00:46<00:00,  6.39it/s]


In [ ]:
df_300 = get_tushare_data(token_poor,'index_daily',ts_code='399300.SZ', start_date='20190101', end_date='20210101')
df_300.to_sql('399300',con,if_exists='append',index=False)